# Master Metrics for L2 Retrospecive Grants

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ramshreyas/retrospective_grants_metrics/blob/main/master_metrics.ipynb)

---

### Setup & Imports

Installs

In [79]:
!pip install pandas

Imports

In [80]:
from google.colab import files
import pandas as pd
import io
from datetime import datetime
import requests
import json
from IPython.display import display, JSON, clear_output
from pprint import pprint
from time import sleep
import warnings
warnings.filterwarnings('ignore')

Repo owner and name extraction function

In [81]:
def extract_github_owner_repo(df, url_column):
    results = []
    for url in df[url_column]:
        # Check if the URL is a string and contains the expected 'github.com'
        if isinstance(url, str) and 'github.com/' in url:
            # Split the URL to isolate the owner and repo
            parts = url.split('github.com/')[-1].split('/')
            if len(parts) >= 2:
                owner, repo = parts[0], parts[1]
                results.append((owner, repo))
            else:
                # Handle cases where URL might not correctly split into owner/repo
                results.append((parts[0], None))
        else:
            # Append None or placeholders if URL is not valid or missing
            results.append((None, None))
    return results

GraphQL Query Function

In [82]:
def execute_graphql_query(query, variables=None, token="your_github_token_here"):
    """
    Executes a GraphQL query to the GitHub API.

    Args:
    query (str): The GraphQL query to execute.
    variables (dict, optional): Variables required for the query. Defaults to None.
    token (str): GitHub API token for authentication.

    Returns:
    dict: The JSON response from the API.
    """
    # Endpoint for GitHub's GraphQL API
    url = 'https://api.github.com/graphql'

    # Headers including the authentication token
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Package the request
    payload = json.dumps({
        'query': query,
        'variables': variables
    })

    # Make the HTTP POST request
    response = requests.post(url, headers=headers, data=payload)

    # Check if the response was successful
    if response.status_code == 200:
        response_data = response.json()  # Parse JSON response
        display(JSON(response_data))  # Display the response in JSON format for clarity
        return response_data
    else:
        print(f"Query failed with status code {response.status_code}: {response.text}")
        return None

HTTP Query Function

In [83]:
def execute_http_query(url, method='GET', headers=None, data=None, params=None, token=None):
    """
    Executes a generic HTTP request to the GitHub API.

    Args:
    url (str): The full URL to the GitHub API endpoint.
    method (stra): The HTTP method to use ('GET', 'POST', etc.).
    headers (dict, optional): Additional HTTP headers as a dictionary.
    data (dict, optional): Data to send with the request (for POST, PUT methods).
    params (dict, optional): URL parameters to append to the request.
    token (str, optional): GitHub API token for authentication.

    Returns:
    tuple: (response content as JSON if available, headers of the response)
    """
    # Initialize headers if not provided
    if headers is None:
        headers = {}
    # Add the authorization token to headers if provided
    if token:
        headers['Authorization'] = f'token {token}'
    headers.setdefault('Accept', 'application/vnd.github.v3+json')

    try:
        # Make the HTTP request
        response = requests.request(method, url, headers=headers, json=data, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        return response.json(), response.headers
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code} {e.response.reason}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError:  # Includes simplejson.decoder.JSONDecodeError
        return response.text, response.headers  # Return plain text if JSON decoding fails

Pull Requests

In [84]:
def fetch_total_pull_requests(owner, repo, token):
    """
    Fetches the total number of pull requests for a given repository.

    Args:
    owner (str): GitHub repository owner.
    repo (str): GitHub repository name.
    token (str): GitHub API token.

    Returns:
    int: Total number of pull requests.
    """
    base_url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
    params = {'per_page': 100, 'state': 'all'}  # Fetch all PRs, both open and closed
    total_pull_requests = 0
    while True:
        response, headers = execute_http_query(base_url, params=params, token=token)
        total_pull_requests += len(response)
        if 'next' in headers.get('Link', ''):
            next_link = headers['Link']
            # Extract the URL for the next page from the 'Link' header
            next_url = next_link.split(';')[0].strip('<>')
            base_url = next_url  # Update the URL to the next page's URL
        else:
            break

    return total_pull_requests

Issues

In [85]:
def get_issues(owner, repo, token):
  query = """
  query GetRepositoryInfo($owner: String!, $repo: String!) {
    repository(owner: $owner, name: $repo) {
      all:issues {
        totalCount
      }
      closed:issues(states:CLOSED) {
        totalCount
      }
      open:issues(states:OPEN) {
        totalCount
      }
    }
  }
  """

  variables = {
      "owner": owner,
      "repo": repo
  }

  # Execute the query
  response = execute_graphql_query(query, variables, token)

  all = response["data"]["repository"]["all"]["totalCount"]
  closed = response["data"]["repository"]["closed"]["totalCount"]
  open = response["data"]["repository"]["open"]["totalCount"]

  return all, closed, open

Issue-Resolution turnaround time

In [86]:
def calculate_average_turnaround(owner, repo, token):
    # Query
    query = """
    query GetIssueTurnaroundTimes($owner: String!, $repo: String!) {
      repository(owner: $owner, name: $repo) {
        issues(first: 100) {  # Adjust 'first' for pagination
          edges {
            node {
              createdAt
              closedAt
            }
          }
          pageInfo {
            endCursor
            hasNextPage
          }
        }
      }
    }
    """

    variables = {
          "owner": owner,
          "repo": repo
      }

    # Execute the query
    data = execute_graphql_query(query, variables, token)

    times = []
    for edge in data['data']['repository']['issues']['edges']:
        if edge['node']['closedAt']:
            start = datetime.fromisoformat(edge['node']['createdAt'].replace('Z', '+00:00'))
            end = datetime.fromisoformat(edge['node']['closedAt'].replace('Z', '+00:00'))
            delta = end - start
            times.append(delta.total_seconds())

    average_seconds = sum(times) / len(times) if times else 0

    return average_seconds / 86400  # Convert seconds to days

---

### Load Grantee Sheet

In [87]:
# Create and display the file upload widget
uploaded_files = files.upload()

Saving L2 Community Grants Retrospective - L2 Retro Opportunity_1.csv to L2 Community Grants Retrospective - L2 Retro Opportunity_1 (2).csv


Check if upload is ok

In [88]:
filename, content = next(iter(uploaded_files.items()))
df = pd.read_csv(io.BytesIO(content))
metrics_df = df[['Opportunity Name', 'Account Name', 'Github']]
metrics_df['Start Date'] = None
metrics_df.head()

,Opportunity Name,Account Name,Github,Start Date
0,Starksheet,Starksheet,https://github.com/the-candy-shop/starksheet-m...,None
1,MyFirstLayer2,LXDAO,https://github.com/lxdao-official/myfirstlayer...,None
2,ERC-4337 Public Infrastructure,Candide,github.com/candidelabs/,None
3,Rollup Security Framework,"Quantstamp, Inc.",https://github.com/quantstamp/l2-security-fram...,None
4,Back-End API Standard for L2 Block Explorers,"Quantstamp, Inc.",https://github.com/quantstamp/l2-block-explore...,None


---

### Generate Metrics

In [89]:
token = ""

In [90]:
owner_repo_list = extract_github_owner_repo(metrics_df, 'Github')

In [100]:
def generate_metrics(owner_repo_list):
  generic_data = []
  stars = []
  forks = []
  pulls = []
  issues = []
  issue_tots = []
  licenses = []

  for owner, repo in owner_repo_list:
    if repo:
      print(f"Fetching generic data for {owner}/{repo}")
      gd = execute_http_query(f'https://api.github.com/repos/{owner}/{repo}', token=token)[0]
      generic_data.append(gd)
      stars.append(gd["stargazers_count"])
      forks.append(gd["forks_count"])
      licenses.append(gd["license"])
      sleep(0.1)
      print(f"Fetching pull data for {owner}/{repo}")
      pulls.append(fetch_total_pull_requests(owner, repo, token))
      sleep(0.1)
      print(f"Fetching issue data for {owner}/{repo}")
      issues.append(get_issues(owner, repo, token))
      sleep(0.1)
      print(f"Calculating issue turnaround time for {owner}/{repo}")
      issue_tots.append(calculate_average_turnaround(owner, repo, token))
      sleep(0.1)
    else:
      print(f"No repo listed for {owner}")
      generic_data.append(None)
      stars.append(None)
      forks.append(None)
      pulls.append(None)
      issues.append(None)
      issue_tots.append(None)
      licenses.append(None)

    clear_output(wait=True)

  return generic_data, stars, forks, pulls, issues, issue_tots, licenses

In [101]:
generic_data, stars, forks, pulls, issues, issue_tots, licenses = generate_metrics(owner_repo_list)

No repo listed for None


In [102]:
metrics_df["stars"] = stars
metrics_df["forks"] = forks
metrics_df["pulls"] = pulls
metrics_df["issues"] = issues
metrics_df["issue_tots"] = issue_tots
metrics_df["license"] = licenses

In [103]:
metrics_df

,Opportunity Name,Account Name,Github,Start Date,stars,forks,pulls,issue_tots,license,issues
0,Starksheet,Starksheet,https://github.com/the-candy-shop/starksheet-m...,None,44.0,11.0,78.0,19.933661,None,"(11, 11, 0)"
1,MyFirstLayer2,LXDAO,https://github.com/lxdao-official/myfirstlayer...,None,15.0,6.0,147.0,4.842865,None,"(18, 8, 10)"
2,ERC-4337 Public Infrastructure,Candide,github.com/candidelabs/,None,NaN,NaN,NaN,NaN,None,None
3,Rollup Security Framework,"Quantstamp, Inc.",https://github.com/quantstamp/l2-security-fram...,None,41.0,6.0,1.0,0.000000,None,"(0, 0, 0)"
4,Back-End API Standard for L2 Block Explorers,"Quantstamp, Inc.",https://github.com/quantstamp/l2-block-explore...,None,11.0,1.0,0.0,0.000000,None,"(0, 0, 0)"
5,L2 en Español,L2 en Espanol,NaN,None,NaN,NaN,NaN,NaN,None,None
6,Layer 2 Activity Tracking & Comparison Suite,growthepie / orbal GmbH,https://github.com/growthepie,None,NaN,NaN,NaN,NaN,None,None
7,Layer1.5,Test In Prod,https://github.com/testinprod-io,None,NaN,NaN,NaN,NaN,None,None
8,Performance and Security Evaluation of Layer 2...,EPFL,NaN,None,NaN,NaN,NaN,NaN,None,None
9,Layer 2 Governance with Flexible Voting,Scopelift,https://github.com/ScopeLift/l2-flexible-voting,None,16.0,2.0,56.0,29.376484,"{'key': 'mit', 'name': 'MIT License', 'spdx_id...","(40, 32, 8)"
